In [77]:
from pymongo import MongoClient
import pandas as pd
from services.db_service import DbService
from datetime import datetime

In [78]:
tournaments = DbService().events_collection()

In [79]:
maindeck_pipeline = [
    {"$match": {"date": {"$gte": datetime.strptime(
        "2024-05-13", "%Y-%m-%d")}}},
    {"$project": {"decklists": "$players.decklist"}},
    {"$unwind": "$decklists"},
    {"$group": {"_id": "$decklists"}},
    {"$lookup": {"from": "decklists", "localField": "_id",
                 "foreignField": "_id", "as": "decklists"}},
    {"$unwind": "$decklists"},
    {"$match": {"decklists.banned": {"$ne": True},
                "decklists.name": {"$eq": "Boros Synthesizer"}}},
    {"$project": {"_id": "$decklists._id", "name": "$decklists.name",
                  "maindeck": "$decklists.maindeck", "sideboard": "$decklists.sideboard"}},
    {"$unwind": "$maindeck"},
    {"$group": {"_id": {"_id": "$_id", "card": "$maindeck"}, "quantity": {"$sum": 1}}},
    {"$project": {"_id": "$_id._id", "card": "$_id.card", "quantity": 1}},
    {"$group": {"_id": "$_id", "quantities": {
        "$push": {"card": "$card", "quantity": "$quantity"}}}},
]

sideboard_pipeline = [
    {"$match": {"date": {"$gte": datetime.strptime(
        "2024-05-13", "%Y-%m-%d")}}},
    {"$project": {"decklists": "$players.decklist"}},
    {"$unwind": "$decklists"},
    {"$group": {"_id": "$decklists"}},
    {"$lookup": {"from": "decklists", "localField": "_id",
                 "foreignField": "_id", "as": "decklists"}},
    {"$unwind": "$decklists"},
    {"$match": {"decklists.banned": {"$ne": True},
                "decklists.name": {"$eq": "Boros Synthesizer"}}},
    {"$project": {"_id": "$decklists._id", "name": "$decklists.name",
                  "maindeck": "$decklists.maindeck", "sideboard": "$decklists.sideboard"}},
    {"$unwind": "$sideboard"},
    {"$group": {"_id": {"_id": "$_id", "card": "$sideboard"}, "quantity": {"$sum": 1}}},
    {"$project": {"_id": "$_id._id", "card": "$_id.card", "quantity": 1}},
    {"$group": {"_id": "$_id", "quantities": {
        "$push": {"card": "$card", "quantity": "$quantity"}}}},
]

In [80]:
maindeck = list(tournaments.aggregate(maindeck_pipeline))
sideboard = list(tournaments.aggregate(sideboard_pipeline))

In [81]:
data = {}
for i in range(len(maindeck)):
    x = maindeck[i]
    # id = str(x["_id"])
    quantities = x["quantities"]

    for item in quantities:
        card = item["card"]
        quantity = item["quantity"]

        if card not in data:
            data[card] = {}
        data[card][i+1] = quantity

In [82]:
df = pd.DataFrame(data)
# .fillna(0).astype(int)
df = df.T

df.index.name = 'Card'
df.columns.name = 'Deck'

df = df.sort_index(axis=1)
df = df.sort_index(axis=0)

df

Deck,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
Card,,,,,,,,,,,,,,,,,,,,,
Abraded Bluffs,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,NaN,4.0,NaN,NaN,NaN
Ancient Den,2.0,2.0,NaN,3.0,2.0,4.0,3.0,2.0,2.0,3.0,...,4.0,4.0,4.0,1.0,4.0,4.0,3.0,NaN,3.0,4.0
Ardent Recruit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
Barbed Batterfist,NaN,2.0,1.0,1.0,4.0,2.0,3.0,3.0,3.0,2.0,...,3.0,3.0,NaN,1.0,2.0,2.0,NaN,1.0,1.0,3.0
Basilisk Gate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
Boros Garrison,2.0,NaN,2.0,1.0,2.0,NaN,NaN,2.0,2.0,NaN,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
Boros Guildgate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
Candy Trail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Chain Lightning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN


In [83]:
stats_df = pd.DataFrame({
    'Min': df.min(axis=1),
    'Max': df.max(axis=1),
    'Mean': df.mean(axis=1).round(2),
    'Median': df.median(axis=1).round(0),
    'Count': df[df > 0].count(axis=1)
})

stats_df = stats_df.sort_values(by='Mean')

stats_df

,Min,Max,Mean,Median,Count
Card,,,,,
Guardian of the Guildpact,1.0,1.0,1.00,1.0,3
Lorehold Campus,1.0,1.0,1.00,1.0,1
Candy Trail,1.0,1.0,1.00,1.0,1
Goblin Grenade,1.0,1.0,1.00,1.0,1
Makeshift Munitions,1.0,2.0,1.05,1.0,21
Krark-Clan Shaman,1.0,2.0,1.33,1.0,3
Dawnbringer Cleric,1.0,2.0,1.36,1.0,11
Goblin Bushwhacker,1.0,2.0,1.50,2.0,2
Boros Garrison,1.0,2.0,1.70,2.0,10


In [84]:
df = df.T

df.index.name = 'Deck'
df.columns.name = 'Card'

df = df.sort_index(axis=1)
df = df.sort_index(axis=0)

df

Card,Abraded Bluffs,Ancient Den,Ardent Recruit,Barbed Batterfist,Basilisk Gate,Boros Garrison,Boros Guildgate,Candy Trail,Chain Lightning,Citadel Gate,...,Outlaw Medic,Plains,Relic of Progenitus,Rustvale Bridge,Sacred Cat,Seeker of the Way,Thraben Inspector,Tormod's Crypt,Wind-Scarred Crag,You Hear Something on Watch
Deck,,,,,,,,,,,,,,,,,,,,,
1,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,3.0,3.0,4.0,NaN,NaN,4.0,NaN,3.0,NaN
2,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,4.0,NaN,3.0,4.0,NaN,4.0,NaN
3,4.0,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,6.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN
4,NaN,3.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,4.0,NaN,1.0,4.0,3.0,3.0,NaN
5,NaN,2.0,NaN,4.0,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,4.0,3.0,4.0,NaN,NaN,4.0,NaN,3.0,NaN
6,NaN,4.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,4.0,NaN,2.0,4.0,NaN,3.0,NaN
7,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,2.0,3.0,NaN,NaN,4.0,NaN,4.0,NaN
8,4.0,2.0,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,4.0,3.0,3.0,NaN,NaN,4.0,NaN,NaN,NaN
9,4.0,2.0,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,4.0,NaN,2.0,4.0,NaN,NaN,NaN


In [85]:
def save_csv_with_commas(df, filename):
    df.to_csv(filename, decimal=',', quoting=1, quotechar='"')

# Salvar o DataFrame com as quantidades em um arquivo CSV
save_csv_with_commas(df, 'deck_quantities.csv')

# Salvar o DataFrame com as estatísticas em um arquivo CSV
save_csv_with_commas(stats_df, 'deck_stats.csv')